In [2]:
import polars as pl
from pprint import pprint

In [3]:
def fix_str_to_int(data:pl.DataFrame, fix_list:list, type:any=pl.Int64):
    data = data.with_columns(pl.col(fix_list).str.split(","))
    data = data.with_columns(pl.col(fix_list).list.eval(pl.element().str.strip_chars()))
    data = data.with_columns(pl.col(fix_list).list.eval(pl.element().cast(pl.Int64, strict=False)))

    for column in fix_list:
        data = data.with_columns(pl.when(pl.col(column).list.eval(pl.element().is_null()).list.any()).then(None).otherwise(pl.col(column)).alias("bools"))
        data = data.select(replace(data, data.get_column_index(column), "bools")).rename({"bools":column})
    
    # data = data.with_columns(pl.col(fix_list).str.strip().str.replace("","0")).cast(conversion_dict)

    return data


# Abych nebyl nařčen z plagiátorství, zdroj: https://stackoverflow.com/questions/75954280/how-to-change-the-position-of-a-single-column-in-python-polars-library
def replace(df, new_position, col_name):
    neworder=df.columns
    neworder.pop(new_position)
    neworder.remove(col_name)
    neworder.insert(new_position,col_name)
    return neworder


In [4]:
rozvrh_by_kat = pl.read_csv("source_tables/rozvrhy_fakulta.csv").drop("semestr").unique().rename({"predmet" : "zkratka"})
rozvrh_by_kat = fix_str_to_int(rozvrh_by_kat, ["ucitIdno.ucitel", "vsichniUciteleUcitIdno"])
#print(rozvrh_by_kat.head())
predmety_by_kat = pl.read_csv("source_tables/predmety_fakulta.csv").drop("semestr").unique().drop(["pocetStudentu", "aSkut", "bSkut", "cSkut"])
predmety_by_kat = fix_str_to_int(predmety_by_kat, ["garantiUcitIdno", "prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno", "hodZaSemKombForma"])
predmety_by_kat.filter(pl.col("zkratka") == "P522").head()
#predmety_by_kat = pl.read_excel("source_tables/getPredmetyByKatedraFullInfo-2023-03-15-17-40.xlsx", sheet_id=1, engine="openpyxl").drop("semestr").unique().drop(["pocetStudentu", "aSkut", "bSkut", "cSkut"])

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KCH""","""P522""",2023,"""Odborný seminá…","""Odborný seminá…","""A""","""A""",1,"""NE""","""""","""'doc. Ing. Jan…","""'doc. Ing. Jan…",[1064],"""""","""""",null,"""'Ing. Ivana Ka…","""'Ing. Ivana Ka…","[4226, 429]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Seminář má umo…","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KBI""","""P522""",2023,"""Fyziologie živ…","""Fyziologie živ…","""A""","""N""",7,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…",[8024],"""'Mgr. Marian R…","""'Mgr. Marian R…","[5275, 8024]","""'Mgr. Marian R…","""'Mgr. Marian R…","[5275, 8024]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'http://biolog…","""""","""Monologická (v…","""Písemná zkoušk…","""N""",2,"""HOD/TYD""",3,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Zápočet: Ne ví…","""1. Vnitřní pro…","""Znalost biolog…","""""","""""","""""","""Čeština, Angli…","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KCH""","""P522""",2023,"""Odborný seminá…","""Odborný seminá…","""A""","""A""",1,"""NE""","""""","""'doc. Ing. Jan…","""'doc. Ing. Jan…",[1064],"""""","""""",null,"""'Ing. Ivana Ka…","""'Ing. Ivana Ka…","[4226, 429]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Seminář má umo…","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KFY""","""P522""",2023,"""Vybrané partie…","""Vybrané partie…","""A""","""A""",4,"""NE""","""""","""'doc. RNDr. In…","""'doc. RNDr. In…",[606],"""'doc. RNDr. In…","""'doc. RNDr. In…",[606],"""'doc. RNDr. In…","""'doc. RNDr. In…",[606],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Feynman R.P.,…","""""","""""","""""","""A""",2,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Kurz je věnová…","""Zkouška""","""ANO""","""Kombinovaná""","""""","""1. Termodynami…","""Výuka v anglič…","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""N""","""""",0,"""""","""Mgr.""","""A""",null,"""""","""""",""""""


In [5]:
# qr = ~pl.all().is_duplicated()
# rozvrh_by_kat = rozvrh_by_kat.filter(qr)
# predmety_by_kat = predmety_by_kat.filter(qr)


In [41]:
predmety_by_kat.filter(pl.col("zkratka") == "B317").head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KCH""","""B317""",2023,"""Membránové sep…","""Membránové sep…","""A""","""A""",3,"""NE""","""""","""'Ing. Zuzana P…","""'Ing. Zuzana P…",[3705],"""'Ing. Zuzana P…","""'Ing. Zuzana P…",[3705],"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""KCH/B104""","""""","""""","""""","""""","""""","""""","""N""",2,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Prezentace vyb…","""1. Historie me…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KCH""","""B317""",2023,"""Membránové sep…","""Membránové sep…","""A""","""A""",3,"""NE""","""""","""'Ing. Zuzana P…","""'Ing. Zuzana P…",[3705],"""'Ing. Zuzana P…","""'Ing. Zuzana P…",[3705],"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""KCH/B104""","""""","""""","""""","""""","""""","""""","""N""",2,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Prezentace vyb…","""1. Historie me…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""


In [40]:
rozvrh_by_kat.head(5)
#rozvrh_by_kat.__len__()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
i64,str,str,str,str,str,list[i64],str,str,str,str,str,str,str,i64,str,str,str,i64,i64,str,str,str,str,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,list[i64],str,str,str,i64,str,str,str,str
491547,"""Membránové sep…","""KCH""","""B317""","""""","""3705""",[3705],"""Zuzana""","""Petrusová""","""Ing.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""3.23""","""28""",5,2,"""Přednáška""","""Př""","""A""","""Pátek""","""Pá""","""""","""6""","""9""",4,"""12:00""","""15:50""",12,12,"""Jiný""","""J""","""32022""","""N""","""N""","""""","""""","""UJEP""","""""","""22.3.2024""","""22.3.2024""","""R""",[3705],"""Ing. Zuzana Pe…","""'Ing. Zuzana P…","""Petrusová""",491547,"""""","""""","""RKADLECKOVA""",""""""
491550,"""Membránové sep…","""KCH""","""B317""","""""","""3705""",[3705],"""Zuzana""","""Petrusová""","""Ing.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""3.23""","""28""",5,2,"""Přednáška""","""Př""","""A""","""Pátek""","""Pá""","""""","""6""","""9""",4,"""12:00""","""15:50""",18,18,"""Jiný""","""J""","""32022""","""N""","""N""","""""","""""","""UJEP""","""""","""3.5.2024""","""3.5.2024""","""R""",[3705],"""Ing. Zuzana Pe…","""'Ing. Zuzana P…","""Petrusová""",491550,"""""","""""","""RKADLECKOVA""",""""""
491549,"""Membránové sep…","""KCH""","""B317""","""""","""3705""",[3705],"""Zuzana""","""Petrusová""","""Ing.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""3.23""","""28""",5,2,"""Přednáška""","""Př""","""A""","""Pátek""","""Pá""","""""","""6""","""9""",4,"""12:00""","""15:50""",17,17,"""Jiný""","""J""","""32022""","""N""","""N""","""""","""""","""UJEP""","""""","""26.4.2024""","""26.4.2024""","""R""",[3705],"""Ing. Zuzana Pe…","""'Ing. Zuzana P…","""Petrusová""",491549,"""""","""""","""RKADLECKOVA""",""""""
491548,"""Membránové sep…","""KCH""","""B317""","""""","""3705""",[3705],"""Zuzana""","""Petrusová""","""Ing.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""3.23""","""28""",5,2,"""Přednáška""","""Př""","""A""","""Pátek""","""Pá""","""""","""6""","""9""",4,"""12:00""","""15:50""",16,16,"""Jiný""","""J""","""32022""","""N""","""N""","""""","""""","""UJEP""","""""","""19.4.2024""","""19.4.2024""","""R""",[3705],"""Ing. Zuzana Pe…","""'Ing. Zuzana P…","""Petrusová""",491548,"""""","""""","""RKADLECKOVA""",""""""
491546,"""Membránové sep…","""KCH""","""B317""","""""","""3705""",[3705],"""Zuzana""","""Petrusová""","""Ing.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""3.23""","""28""",5,2,"""Přednáška""","""Př""","""A""","""Pátek""","""Pá""","""""","""6""","""9""",4,"""12:00""","""15:50""",10,10,"""Jiný""","""J""","""32022""","""N""","""N""","""""","""""","""UJEP""","""""","""8.3.2024""","""8.3.2024""","""R""",[3705],"""Ing. Zuzana Pe…","""'Ing. Zuzana P…","""Petrusová""",491546,"""""","""""","""RKADLECKOVA""",""""""


In [8]:
set(rozvrh_by_kat.columns) & set(predmety_by_kat.columns)

{'katedra', 'nazev', 'rok', 'zkratka'}

In [9]:
rozvrh_by_kat.null_count()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,164,238,0,0,0,0,0,0,164,0,0,66,160,0,0,0,0,0,0,0,0,474,474,171,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,238,0,0,0,0,0,0,0,0


In [10]:
predmety_by_kat.null_count()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,391,0,0,1700,0,0,1306,0,0,2798,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1931,0,0,0


In [11]:
# vybírám pouze předměty, které mají rozvrhovou akci

In [12]:
predmety_s_akci = predmety_by_kat.join(other=rozvrh_by_kat, on="zkratka", how="inner").select(predmety_by_kat.columns).unique().sort("zkratka")
str(len(predmety_s_akci)) + "   " + str(len(predmety_by_kat)) 
predmety_s_akci.filter(pl.col("zkratka") == "P522").head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KBI""","""P522""",2023,"""Fyziologie živ…","""Fyziologie živ…","""A""","""N""",7,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…",[8024],"""'Mgr. Marian R…","""'Mgr. Marian R…","[5275, 8024]","""'Mgr. Marian R…","""'Mgr. Marian R…","[5275, 8024]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'http://biolog…","""""","""Monologická (v…","""Písemná zkoušk…","""N""",2,"""HOD/TYD""",3,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Zápočet: Ne ví…","""1. Vnitřní pro…","""Znalost biolog…","""""","""""","""""","""Čeština, Angli…","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KFY""","""P522""",2023,"""Vybrané partie…","""Vybrané partie…","""A""","""A""",4,"""NE""","""""","""'doc. RNDr. In…","""'doc. RNDr. In…",[606],"""'doc. RNDr. In…","""'doc. RNDr. In…",[606],"""'doc. RNDr. In…","""'doc. RNDr. In…",[606],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Feynman R.P.,…","""""","""""","""""","""A""",2,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Kurz je věnová…","""Zkouška""","""ANO""","""Kombinovaná""","""""","""1. Termodynami…","""Výuka v anglič…","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""N""","""""",0,"""""","""Mgr.""","""A""",null,"""""","""""",""""""
"""KCH""","""P522""",2023,"""Odborný seminá…","""Odborný seminá…","""A""","""A""",1,"""NE""","""""","""'doc. Ing. Jan…","""'doc. Ing. Jan…",[1064],"""""","""""",null,"""'Ing. Ivana Ka…","""'Ing. Ivana Ka…","[4226, 429]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Seminář má umo…","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""


# Vyučující a garanti

In [13]:
jednotek_prednasek = pl.col("jednotekPrednasek")
jednotek_cviceni = pl.col("jednotekCviceni")
jednotek_seminare = pl.col("jednotekSeminare")
garant = pl.col("garanti")
cvicici = pl.col("cvicici")
prednasejici = pl.col("prednasejici")
seminarici = pl.col("seminarici")

### Každý předmět má právě jednoho garanta

> Zde jsou předměty, které nemají garanta:

In [14]:
zkratky = predmety_s_akci.filter(pl.col("garantiUcitIdno").is_null()).select(["zkratka", "rok", "nazev", "nazevDlouhy", "garanti", "garantiSPodily"]).filter(
    # Není SZ
    # pl.col("zkratka").str.starts_with("SZ").is_not()
    True
)
print(f"\n{len(zkratky)} Předmětů nemá zadaného garanta")
zkratky.write_excel("results/bez_garanta.xlsx")
zkratky
# print(list(zkratky), sep=", ")


163 Předmětů nemá zadaného garanta


zkratka,rok,nazev,nazevDlouhy,garanti,garantiSPodily
str,i64,str,str,str,str
"""0099""",2023,"""VK - Geodetick…","""Geodetické met…","""""",""""""
"""0121""",2023,"""VK-Propagace a…","""Propagace a pr…","""""",""""""
"""0165""",2023,"""Hodnotová orie…","""Hodnotová orie…","""""",""""""
"""0199""",2023,"""VK - Aplikace …","""Aplikace ATMEL…","""""",""""""
"""0201""",2023,"""VK-Praktikum m…","""VK-Praktikum m…","""""",""""""
…,…,…,…,…,…
"""SZZSŠ""",2023,"""Učitelství inf…","""Učitelství inf…","""""",""""""
"""SZZSŠ""",2023,"""Učitelství mat…","""Učitelství mat…","""""",""""""
"""SZZTD""",2023,"""Teoretické zák…","""Teoretické zák…","""""",""""""


> Předměty podezřelé z více garantů:

In [15]:
# predmety_s_akci.filter( > pl.lit())
# Nefunguje, arr.lengths údajně nic není
vice_garantu = predmety_s_akci.with_columns(pl.col("garantiUcitIdno").list.len().alias("pocet garantu")
    ).select(
        ["garantiUcitIdno", "garanti", "zkratka", "pocet garantu"]
    ).filter(pl.col("pocet garantu") > 1)
vice_garantu.write_excel("results/vice_garantu.xlsx")
vice_garantu.head(10)

garantiUcitIdno,garanti,zkratka,pocet garantu
list[i64],str,str,u32
"[315, 3164]","""'RNDr. Ivan Fa…","""0023""",2
"[315, 3164]","""'RNDr. Ivan Fa…","""0065""",2
"[1803, 305]","""'doc. Ing. Jar…","""B201""",2
"[3452, 3528]","""'doc. PhDr. RN…","""B600""",2
"[8441, 283]","""'doc. PhDr. Ka…","""BK107""",2
"[1999, 8280]","""'RNDr. Blanka …","""BK108""",2
"[4380, 5754]","""'Mgr. Jaroslav…","""K100""",2
"[304, 612]","""'doc. RNDr. Ma…","""K206""",2
"[3661, 283]","""'prof. doc. RN…","""K301""",2


### Každý předmět má alespoň jednoho vyučujícího v kategoriích dle parametrů předmětu

např. pokud má předmět rozsah 2 h přednášek a 2 h cvičení, pak musí mít alespoň jednoho přednášejícího a alespoň jednoho cvičícího a zároveň nesmí mít vyučujícího v kategorii vede seminář.

In [16]:
ma_prednasku = jednotek_prednasek != 0
nema_prednasejiciho = pl.col("prednasejiciUcitIdno").is_null()

chybi_prednasejici = predmety_s_akci.filter(ma_prednasku & nema_prednasejiciho)
chybi_prednasejici.write_excel("results/chybi_prednasejici.xlsx")
chybi_prednasejici.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KPRF""","""0101""",2023,"""Úvod do meteor…","""Úvod do meteor…","""A""","""N""",3,"""NE""","""""","""'doc. Mgr. Pav…","""'doc. Mgr. Pav…",[1434],"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""KPRF/0102""","""""","""""","""""","""""","""N""",2,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Meteorologie j…","""Zápočet""","""NE""","""Kombinovaná""","""""","""1. Historie, s…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KGEO""","""0159""",2023,"""Smart City""","""Smart City""","""A""","""A""",2,"""NE""","""""","""'Mgr. Vladan H…","""'Mgr. Vladan H…",[4222],"""""","""""",null,"""'Mgr. Vladan H…","""'Mgr. Vladan H…","[4222, 7684]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Vanolo, A. (2…","""""","""""","""""","""A""",1,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Smart city/reg…","""Zápočet""","""NE""","""Kombinovaná""","""prezence na cv…","""1. Konceptuali…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KGEO""","""B210""",2023,"""Euroregiony a …","""Euroregiony a …","""N""","""A""",4,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'JEŘÁBEK, M., …","""""","""""","""""","""N""",2,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem předmětu…","""Zkouška""","""ANO""","""Kombinovaná""","""- odevzdání vš…","""1. Národ, náro…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""12""","""3""","""5"""
"""KGEO""","""B414""",2023,"""Ekonomická geo…","""Ekonomická geo…","""N""","""A""",4,"""NE""","""""","""'Mgr. Vladan H…","""'Mgr. Vladan H…",[4222],"""""","""""",null,"""'Mgr. Vladan H…","""'Mgr. Vladan H…",[4222],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Coe, N., Kell…","""""","""""","""""","""N""",2,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Kurz řeší prob…","""Zápočet""","""NE""","""Kombinovaná""","""Aktivní účast …","""1. Komoditní ř…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""20""","""10""","""1"""
"""KBI""","""BK201""",2023,"""Chemie a fyzik…","""Chemie a fyzik…","""A""","""N""",4,"""NE""","""""","""'RNDr. Oldřich…","""'RNDr. Oldřich…",[1983],"""""","""""",null,"""""","""""",nul

In [17]:
ma_cviceni = jednotek_cviceni != 0
nema_cviciciho = pl.col("cviciciUcitIdno").is_null()

chybi_cvicici = predmety_s_akci.filter(ma_cviceni & nema_cviciciho)
chybi_cvicici.write_excel("results/chybi_cvicici.xlsx")
chybi_cvicici.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KGEO""","""0099""",2023,"""VK - Geodetick…","""Geodetické met…","""A""","""N""",3,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Vinkler. F. G…","""""","""""","""""","""A""",0,"""HOD/TYD""",3,"""HOD/TYD""",0,"""HOD/TYD""","""""","""Zápočet""","""NE""","""Kombinovaná""","""Aktivní účast …","""Tematické zamě…","""""","""""","""""","""""","""Čeština""","""vk""","""N""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KGEO""","""0121""",2023,"""VK-Propagace a…","""Propagace a pr…","""A""","""A""",2,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'CLOW K., E., …","""""","""""","""""","""N""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""Kurz je zaměře…","""Zápočet""","""NE""","""Kombinovaná""","""výborná práce …","""seznámením s m…","""""","""""","""""","""""","""Čeština""","""""","""N""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KGEO""","""0165""",2023,"""Hodnotová orie…","""Hodnotová orie…","""A""","""A""",2,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Patočka, J. D…","""""","""""","""""","""A""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""",""" Kurz je konci…","""Zápočet""","""NE""","""Kombinovaná""","""1. Aktivní úča…","""1. Přehled dis…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KFY""","""0199""",2023,"""VK - Aplikace …","""Aplikace ATMEL…","""A""","""A""",2,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""KFY/0174""","""""","""""","""A""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""""","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KFY""","""0201""",2023,"""VK-Praktikum m…","""VK-Praktikum m…","""A""","""A""",2,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""A""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""""","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""

In [18]:
ma_seminare = jednotek_seminare != 0
nema_seminariciho = pl.col("seminariciUcitIdno").is_null()

chybi_seminarici = predmety_s_akci.filter(ma_seminare & nema_seminariciho)
chybi_seminarici.write_excel("results/chybi_seminarici.xlsx")
chybi_seminarici

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KCH""","""BK316""",2023,"""Laboratorní cv…","""Laboratorní cv…","""N""","""A""",4,"""NE""","""""","""'RNDr. Thu Huo…","""'RNDr. Thu Huo…",[429],"""""","""""",null,"""'Ing. Ivana Ka…","""'Ing. Ivana Ka…","[4226, 429]","""""","""""",null,"""""","""""","""""","""""","""KCH/BK301""","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/SEM""",16,"""HOD/SEM""",1,"""HOD/SEM""","""Studenti se se…","""Zápočet""","""NE""","""Kombinovaná""","""""","""1. Důkaz příto…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KI""","""KAFJ""",2023,"""Teorie automat…","""Teorie automat…","""A""","""N""",5,"""NE""","""""","""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…","[449, 2220]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Chytil M. Aut…","""""","""""","""""","""N""",6,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""V tomto předmě…","""Zkouška""","""ANO""","""Kombinovaná""","""Požadavky pro …","""1. Konečné aut…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""","""0"""
"""KI""","""KANE""",2023,"""Analogová elek…","""Analogová elek…","""A""","""N""",2,"""NE""","""""","""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""""","""""",null,"""""","""""","""""","""""","""KI/KZEL""","""""","""""","""'Tichý, M. Ele…","""""","""""","""""","""N""",2,"""HOD/SEM""",4,"""HOD/SEM""",2,"""HOD/SEM""","""Kurz je zaměře…","""Zápočet""","""NE""","""Kombinovaná""","""Ústní obhajoba…","""1. Pasivní ele…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KI""","""KAPR1""",2023,"""Algoritmizace …","""Algoritmizace …","""A""","""N""",4,"""NE""","""""","""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…",[251],"""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…",[251],"""'Ing. Mgr. Pav…","""'Ing. Mgr. Pav…","[6973, 650, … 6259]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""KI/KAPR2, KI/K…","""'https://githu…","""""","""""","""""","""N""",6,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""Úvodní kurz al…","""Zápočet""","""NE""","""Kombinovaná""","""Podmínkou získ…","""1. Základní te…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","

> Zde jsou předměty, ve kterých učitel vede seminář:

In [19]:
#prednasejici_je_seminarici = prednasejici == seminarici
#cvicici_je_seminarici = cvicici == seminarici

#konflikt_vyucujicich = predmety_s_akci.filter(ma_seminare & ma_prednasku & (prednasejici_je_seminarici | cvicici_je_seminarici))
#konflikt_vyucujicich.write_excel("results/konflikt_vyucujicich.xlsx")
#konflikt_vyucujicich.head()

In [20]:
print(f"{len(chybi_prednasejici)} předmětů nemá přednášejícího")
print(f"{len(chybi_cvicici)} předmětů nemá cvičícího")
print(f"{len(chybi_seminarici)} předmětů nemá seminářícího")
# print(f"U {len(konflikt_vyucujicich)} předmětů je seminářící také učitel")

31 předmětů nemá přednášejícího
84 předmětů nemá cvičícího
36 předmětů nemá seminářícího


### Garant by měl být někde mezi vyučujícími. Pokud má předmět přednášku, měl by být přednášející.

> V těchto předmětech garant vůbec neučí:

In [21]:
filter_query = (garant != cvicici) & (garant != seminarici) & (garant != prednasejici) & (ma_prednasku | ma_cviceni | ma_seminare)
predmety_kde_garant_neuci = predmety_s_akci.filter(filter_query)
predmety_kde_garant_neuci.write_excel("results/predmety_kde_garant_neuci.xlsx")
predmety_kde_garant_neuci.__len__()

543

> v těchto předmětech je přednáška, ale nedělá jí garant:

In [22]:
garant_neni_prednasejici = (jednotek_prednasek != 0) & (garant != prednasejici)
garant_neprednasi = predmety_s_akci.filter(garant_neni_prednasejici)
garant_neprednasi.write_excel("results/garant_neprednasi.xlsx")
garant_neprednasi.__len__()

307

## Kontrola proti rozvrhovaným akcím

### Vyučující u rozvrhových akcí musí být v příslušných kategoriích u vyučujících

#### Osekám dataframy

In [23]:
rozvrh_by_kat.head()
# predmety_by_kat.head()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
i64,str,str,str,str,str,list[i64],str,str,str,str,str,str,str,i64,str,str,str,i64,i64,str,str,str,str,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,list[i64],str,str,str,i64,str,str,str,str
470891,"""3D tisk v chem…","""CNB""","""M103""","""""","""1503""",[1503],"""Marcel""","""Štofik""","""Mgr.""","""Ph.D.""","""A""","""A""","""50""",2023,"""CP""","""3.18""","""20""",5,4,"""Cvičení""","""Cv""","""A""","""Čtvrtek""","""Čt""","""""","""4""","""5""",2,"""10:00""","""11:50""",40,1,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.N ACH""","""UJEP""","""""","""5.10.2023""","""4.1.2024""","""R""","[1503, 8314]","""Mgr. Marcel Št…","""'Mgr. Marcel Š…","""Štofik, Smejka…",470891,"""""","""""","""PETRUSPRF""",""""""
484583,"""Kompozitní a n…","""CNB""","""KNM""","""""","""4587""",[4587],"""Petr""","""Ryšánek""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""1.03""","""95""",4,3,"""Cvičení""","""Cv""","""A""","""Pondělí""","""Po""","""""","""4""","""4""",1,"""10:00""","""10:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""3.NANO""","""UJEP""","""""","""19.2.2024""","""13.5.2024""","""R""",[4587],"""RNDr. Petr Ryš…","""'RNDr. Petr Ry…","""Ryšánek""",484583,"""""","""""","""RSMEJKAL""",""""""
484586,"""Mikrofluidní s…","""CNB""","""MSBA""","""""","""1503""",[1503],"""Marcel""","""Štofik""","""Mgr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""4.19""","""16""",8,1,"""Cvičení""","""Cv""","""A""","""Čtvrtek""","""Čt""","""""","""2""","""2""",1,"""08:00""","""08:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""2.NANO,3.NANO""","""UJEP""","""""","""22.2.2024""","""16.5.2024""","""R""",[1503],"""Mgr. Marcel Št…","""'Mgr. Marcel Š…","""Štofik""",484586,"""""","""""","""RSMEJKAL""",""""""
489176,"""Molekulární bi…","""CNB""","""BP617""","""""","""8512""",[8512],"""Olga""","""Šebestová Jano…","""Mgr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""""","""""","""""",12,9,"""Cvičení""","""Cv""","""A""","""""","""""","""""","""""","""""",2,"""00:00""","""00:00""",0,0,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""2.BI JO""","""""","""""","""""","""""","""R""",[8512],"""Mgr. Olga Šebe…","""'Mgr. Olga Šeb…","""Šebestová Jano…",489176,"""""","""""","""RSMEJKAL""",""""""
489162,"""Praktický kurz…","""CNB""","""ME202""","""""","""8024""",[8024],"""Stanislav""","""Vinopal""","""Ing.""","""Ph.D.""","""A""","""A""","""100""",2023,"""""","""""","""""",5,5,"""Přednáška""","""Př""","""A""","""""","""""","""""","""""","""""",6,"""00:00""","""00:00""",0,0,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.BI N EB""","""""","""""","""""","""""","""R""","[8024, 280, 1503]","""Ing. Stanislav…","""'Ing. Stanisla…","""Vinopal, Malý,…",489162,"""""","""""","""RSMEJKAL""",""""""


In [35]:
# PROZATÍMNÍ NASTAVENÍ FILTRUJE AKCE BEZ DATUMU

maly_rozvrh = rozvrh_by_kat.select(["katedra","zkratka", "vsichniUciteleUcitIdno", "typAkceZkr", "rok", "datumOd", "datumDo", "hodinaSkutOd", "hodinaSkutDo"])
maly_rozvrh = maly_rozvrh.with_columns(pl.col("vsichniUciteleUcitIdno")).explode("vsichniUciteleUcitIdno").rename({"vsichniUciteleUcitIdno": "idno"}).filter(pl.col("datumOd").str.len_chars() > 0)
maly_rozvrh.filter(pl.col("katedra").is_null()).head()

katedra,zkratka,idno,typAkceZkr,rok,datumOd,datumDo,hodinaSkutOd,hodinaSkutDo
str,str,i64,str,i64,str,str,str,str


In [25]:
# predmety_by_kat

In [26]:
from pprint import pprint
# pprint(predmety_by_kat.columns,compact=True)
predmety_by_kat.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""CNB""","""EXK""",2023,"""Exkurze""","""Exkurze""","""A""","""A""",1,"""NE""","""""","""'doc. Ing. Mar…","""'doc. Ing. Mar…",[2384],"""""","""""",null,"""""","""""",null,"""'doc. Ing. Mar…","""'doc. Ing. Mar…",[2384],"""""","""""","""""","""""","""""","""""","""""","""""","""KFY/EXK""","""""","""""","""N""",0,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem kurzu je…","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""CNB""","""NA08""",2023,"""Metody příprav…","""Metody příprav…","""A""","""N""",2,"""NE""","""""","""'doc. RNDr. Ja…","""'doc. RNDr. Ja…",[308],"""'prof. Ing. Zd…","""'prof. Ing. Zd…","[332, 280, 308]","""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""KCH/NA03""","""'Challa S. S. …","""KFY/NA08""","""""","""Písemná zkoušk…","""N""",1,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""""","""Zápočet""","""NE""","""Kombinovaná""","""""","""Předmět bude z…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Mgr.""","""A""",null,"""""","""""",""""""
"""CNB""","""NA22""",2023,"""Úvod do supram…","""Úvod do supram…","""A""","""A""",2,"""NE""","""""","""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'P. Hobza, R. …","""KFY/P701""","""""","""""","""N""",1,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem předmětu…","""Zápočet""","""NE""","""Kombinovaná""","""""","""Osnova: 1. In…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Mgr.""","""A""",null,"""""","""""",""""""
"""CNB""","""BIOM""",2023,"""Biomateriály""","""Biomateriály""","""N""","""A""",2,"""NE""","""""","""'Mgr. Jan Malý…","""'Mgr. Jan Malý…",[280],"""'prof. Ing. Zd…","""'prof. Ing. Zd…","[332, 280, 4711]","""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Williams, D. …","""KBI/BIOM""","""""","""Ústní zkouška""","""N""",1,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět je zam…","""Zkouška""","""NE""","""Kombinovaná""","""""","""1. Úvod do bio…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""CNB""","""KPCSM""",2023,"""Prakt. cvičení…","""Praktická cvič…","""N""","""A""",3,"""N

In [27]:
ciselnik_ucitelu = pl.read_csv("source_tables/ciselnik_ucitelu.csv").select("nazev", "key").rename({"nazev":"jmena", "key":"idno"})
ciselnik_ucitelu.head()

jmena,idno
str,i64
"""Abrahamová Pet…",8409
"""Ač Alexander, …",9316
"""Adamec Slavomí…",5432
"""Adámek Pavel, …",7443
"""Adam Julie, M.…",9249


In [49]:
male_predmety = predmety_by_kat.with_columns(pl.concat_str(pl.col("zkratka"), pl.col("katedra")).alias("zkratka + katedra")).select(pl.col(["zkratka", "katedra", "zkratka + katedra", "rok", "nazev", "garantiUcitIdno", "prednasejici", "prednasejiciUcitIdno","cvicici", "cviciciUcitIdno","seminarici", "seminariciUcitIdno"])).unique(subset="zkratka + katedra").drop("zkratka + katedra")
#male_predmety = male_predmety.with_columns(pl.col(["prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno", "garantiUcitIdno"]).str.split(", "))
male_predmety.filter(pl.col("zkratka") == "B317").head()

zkratka,katedra,rok,nazev,garantiUcitIdno,prednasejici,prednasejiciUcitIdno,cvicici,cviciciUcitIdno,seminarici,seminariciUcitIdno
str,str,i64,str,list[i64],str,list[i64],str,list[i64],str,list[i64]
"""B317""","""KCH""",2023,"""Membránové sep…",[3705],"""'Ing. Zuzana P…",[3705],"""""",null,"""""",null


In [29]:
# přednášející nemá přednášku:
filtrovani_prednasejici = male_predmety.select("nazev", "zkratka", "prednasejiciUcitIdno").explode("prednasejiciUcitIdno")
prednasejici_jmena = male_predmety.select("prednasejici").rename({"prednasejici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
prednasejici_jmena = prednasejici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_prednasejici = filtrovani_prednasejici.with_columns(prednasejici_jmena).filter(
    pl.col("prednasejiciUcitIdno"
).is_not_null()).with_columns(
    pl.col("prednasejiciUcitIdno").alias("idno")
)
joined_prednasejici = filtrovani_prednasejici.join(maly_rozvrh, "idno", "left")
prednasejici_bez_prednasek = joined_prednasejici.filter(pl.col("typAkceZkr").is_null())
prednasejici_bez_prednasek.select("nazev", "zkratka", "prednasejiciUcitIdno", "jmena").sort("prednasejiciUcitIdno").write_excel("results/prednasejici_bez_prednasek.xlsx")
prednasejici_bez_prednasek.select("nazev", "zkratka", "prednasejiciUcitIdno", "jmena").sort("prednasejiciUcitIdno").write_csv("results_testing/prednasejici_bez_prednasek.csv")

In [56]:
#WEIRD STUFF: Učitel 8027 u KGEO/B508
# otherwise looks pretty good

male_prednasky = maly_rozvrh.filter(pl.col("typAkceZkr") == "Př")
joined_prednasky = male_prednasky.join(male_predmety.select("zkratka", "katedra", "nazev", "prednasejici", "prednasejiciUcitIdno"), "zkratka", "left").unique()
prednasky_bez_prednasejicich = joined_prednasky.filter(pl.col("idno").is_in(pl.col("prednasejiciUcitIdno")).not_() & ((pl.col("katedra") == pl.col("katedra_right")) | pl.col("katedra_right").is_null()))

prednasky_bez_prednasejicich = prednasky_bez_prednasejicich.join(ciselnik_ucitelu, "idno", "left").with_columns(pl.col("prednasejiciUcitIdno").cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

prednasky_bez_prednasejicich.write_csv("results_testing/prednasky_bez_prednasejicich.csv")
prednasky_bez_prednasejicich.write_excel("results/prednasky_bez_prednasejicich.xlsx")


print(prednasky_bez_prednasejicich.__len__())
prednasky_bez_prednasejicich.head(15)

115


katedra,zkratka,idno,typAkceZkr,rok,datumOd,datumDo,hodinaSkutOd,hodinaSkutDo,katedra_right,nazev,prednasejici,prednasejiciUcitIdno,jmena
str,str,i64,str,i64,str,str,str,str,str,str,str,str,str
"""KPRF""","""0101""",556,"""Př""",2023,"""2.10.2023""","""18.12.2023""","""16:00""","""17:50""","""KPRF""","""Úvod do meteor…","""""",null,null
"""KCH""","""B100""",2856,"""Př""",2023,"""3.10.2023""","""2.1.2024""","""12:00""","""13:50""","""KCH""","""Úvod do chemie…","""'doc. RNDr. Ja…","""2855""","""Škvorová Magda…"
"""KCH""","""B104""",8421,"""Př""",2023,"""20.2.2024""","""14.5.2024""","""12:00""","""13:50""","""KCH""","""Anorganická ch…","""'RNDr. Václav …","""4310""","""Vojtíšek Pavel…"
"""KGEO""","""B203""",3164,"""Př""",2023,"""21.2.2024""","""15.5.2024""","""17:00""","""18:50""","""KGEO""","""Geografie Česk…","""'doc. Mgr. Pav…","""1434""","""Riezner Jiří, …"
"""KGEO""","""B206""",6384,"""Př""",2023,"""21.2.2024""","""15.5.2024""","""08:00""","""09:50""","""KGEO""","""Regionální geo…","""'doc. RNDr. Ji…","""312""","""Trahorsch Petr…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""KGEO""","""B508""",8027,"""Př""",2023,"""3.10.2023""","""2.1.2024""","""17:00""","""18:50""","""KGEO""","""Ochrana přírod…","""'RNDr. Tomáš M…","""1472""",null
"""KGEO""","""B602""",9521,"""Př""",2023,"""19.2.2024""","""13.5.2024""","""14:00""","""15:50""","""KGEO""","""Specializační …","""'Mgr. Petr Mey…","""5855""","""Vaverka Lukáš,…"
"""KGEO""","""B602""",3528,"""Př""",2023,"""19.2.2024""","""13.5.2024""","""14:00""","""15:50""","""KGEO""","""Specializační …","""'Mgr. Petr Mey…","""5855""","""Kučerová Silvi…"


In [ ]:
# cvičící nemá cvičení:
filtrovani_cvicici = male_predmety.select("nazev", "zkratka", "cviciciUcitIdno").explode("cviciciUcitIdno")
cvicici_jmena = male_predmety.select("cvicici").rename({"cvicici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
cvicici_jmena = cvicici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_cvicici = filtrovani_cvicici.with_columns(cvicici_jmena).filter(
    pl.col("cviciciUcitIdno"
).is_not_null()).with_columns(
    pl.col("cviciciUcitIdno").alias("idno")
)
joined_cvicici = filtrovani_cvicici.join(maly_rozvrh, "idno", "left")
cvicici_bez_cviceni = joined_cvicici.filter(pl.col("typAkceZkr").is_null())
cvicici_bez_cviceni.select("nazev", "zkratka", "cviciciUcitIdno").sort("cviciciUcitIdno").write_excel("results/cvicici_bez_cviceni.xlsx")
cvicici_bez_cviceni.select("nazev", "zkratka", "cviciciUcitIdno", "jmena").sort("cviciciUcitIdno").write_csv("results_testing/cvicici_bez_cviceni.csv")

In [54]:
#WEIRD STUFF: KGEO/0165 má napsaná cvičení (jak v rozvrhu tak předmětu) ale nemá napsanýho žádnýho učitele (fakt ani garanta, ani jednoho přednášejícího/cvičícího/etc.), tf?

male_cviceni = maly_rozvrh.filter(pl.col("typAkceZkr") == "Cv")
joined_cviceni = male_cviceni.join(male_predmety.select("zkratka", "katedra", "nazev", "cvicici", "cviciciUcitIdno"), "zkratka", "left").unique()
cviceni_bez_cvicich = joined_cviceni.filter(pl.col("idno").is_in(pl.col("cviciciUcitIdno")).not_() & ((pl.col("katedra") == pl.col("katedra_right")) | pl.col("katedra_right").is_null()))

cviceni_bez_cvicich = cviceni_bez_cvicich.join(ciselnik_ucitelu, "idno", "left").with_columns(pl.col("cviciciUcitIdno").cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

cviceni_bez_cvicich.write_csv("results_testing/cviceni_bez_cvicich.csv")
cviceni_bez_cvicich.write_excel("results/cviceni_bez_cvicich.xlsx")

print(cviceni_bez_cvicich.__len__())
cviceni_bez_cvicich.head(15)

260


katedra,zkratka,idno,typAkceZkr,rok,datumOd,datumDo,hodinaSkutOd,hodinaSkutDo,katedra_right,nazev,cvicici,cviciciUcitIdno,jmena
str,str,i64,str,i64,str,str,str,str,str,str,str,str,str
"""KGEO""","""0165""",9282,"""Cv""",2023,"""21.2.2024""","""15.5.2024""","""12:00""","""13:50""","""KGEO""","""Hodnotová orie…","""""",null,"""Vyhnánková Kat…"
"""KFY""","""0219""",7723,"""Cv""",2023,"""4.10.2023""","""3.1.2024""","""16:00""","""17:50""","""KFY""","""CFD Simulace""","""'doc. Ing. Jar…","""1803""","""Kramoliš David…"
"""KGEO""","""0A00""",6919,"""Cv""",2023,"""4.10.2023""","""3.1.2024""","""13:00""","""14:50""","""KGEO""","""Diplomová prác…","""'RNDr. Silvie …","""3528""","""Píša Jan, Mgr.…"
"""KCH""","""ACH""",9301,"""Cv""",2023,"""21.2.2024""","""15.5.2024""","""11:00""","""11:50""","""KCH""","""Anorganická ch…","""'RNDr. Václav …","""4310""","""Greguš Viktor,…"
"""KCH""","""B100""",2856,"""Cv""",2023,"""5.10.2023""","""4.1.2024""","""10:00""","""11:50""","""KCH""","""Úvod do chemie…","""'doc. RNDr. Ja…","""2855""","""Škvorová Magda…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""KCH""","""B104""",9301,"""Cv""",2023,"""21.2.2024""","""15.5.2024""","""11:00""","""11:50""","""KCH""","""Anorganická ch…","""'RNDr. Václav …","""4310""","""Greguš Viktor,…"
"""KGEO""","""B104""",8819,"""Cv""",2023,"""5.10.2023""","""4.1.2024""","""10:00""","""11:50""","""KGEO""","""Geografická ka…","""'doc. PhDr. RN…","""3452, 5855""","""Bartůněk Marti…"
"""KCH""","""B107""",4310,"""Cv""",2023,"""2.5.2024""","""2.5.2024""","""13:00""","""13:50""","""KCH""","""Prezentace kat…","""'doc. Ing. Jan…","""1064""","""Šícha Václav, …"


In [ ]:
# seminařicí nemá seminář:
filtrovani_seminarici = male_predmety.select("nazev", "zkratka", "seminariciUcitIdno").explode("seminariciUcitIdno")
seminarici_jmena = male_predmety.select("seminarici").rename({"seminarici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
seminarici_jmena = seminarici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_seminarici = filtrovani_seminarici.with_columns(seminarici_jmena).filter(
    pl.col("seminariciUcitIdno"
).is_not_null()).with_columns(
    pl.col("seminariciUcitIdno").alias("idno")
)
joined_seminarici = filtrovani_seminarici.join(maly_rozvrh, "idno", "left")
seminarici_bez_seminare = joined_seminarici.filter(pl.col("typAkceZkr").is_null())
seminarici_bez_seminare.select("nazev", "zkratka", "seminariciUcitIdno").sort("seminariciUcitIdno").write_excel("results/seminarici_bez_seminare.xlsx")

In [58]:
male_seminare = maly_rozvrh.filter(pl.col("typAkceZkr") == "Se")
joined_seminare = male_seminare.join(male_predmety.select("zkratka", "katedra", "nazev", "seminarici", "seminariciUcitIdno"), "zkratka", "left").unique()
seminare_bez_seminaricich = joined_seminare.filter(pl.col("idno").is_in(pl.col("seminariciUcitIdno")).not_() & ((pl.col("katedra") == pl.col("katedra_right")) | pl.col("katedra_right").is_null()))

seminare_bez_seminaricich = seminare_bez_seminaricich.join(ciselnik_ucitelu, "idno", "left").with_columns(pl.col("seminariciUcitIdno").cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

seminare_bez_seminaricich.write_csv("results_testing/seminare_bez_seminaricich.csv")
seminare_bez_seminaricich.write_excel("results/seminare_bez_seminaricich.xlsx")
print(seminare_bez_seminaricich.__len__())
seminare_bez_seminaricich.head(6)

67


katedra,zkratka,idno,typAkceZkr,rok,datumOd,datumDo,hodinaSkutOd,hodinaSkutDo,katedra_right,nazev,seminarici,seminariciUcitIdno,jmena
str,str,i64,str,i64,str,str,str,str,str,str,str,str,str
"""KI""","""0204""",null,"""Se""",2023,"""27.2.2024""","""7.5.2024""","""15:00""","""16:50""","""KI""","""Microsoft SQL …","""'Mgr. Květuše …","""835""",null
"""KI""","""0204""",null,"""Se""",2023,"""20.2.2024""","""14.5.2024""","""15:00""","""15:50""","""KI""","""Microsoft SQL …","""'Mgr. Květuše …","""835""",null
"""KCH""","""B213""",8946,"""Se""",2023,"""20.2.2024""","""14.5.2024""","""10:00""","""11:50""","""KCH""","""Odborná anglič…","""'doc. Ing. Jan…","""1064""","""Paříková Anna,…"
"""KCH""","""BK213""",8946,"""Se""",2023,"""20.4.2024""","""20.4.2024""","""13:00""","""16:50""","""KCH""","""Odborná anglič…","""'doc. Ing. Jan…","""1064""","""Paříková Anna,…"
"""KCH""","""BK213""",8946,"""Se""",2023,"""6.4.2024""","""6.4.2024""","""13:00""","""16:50""","""KCH""","""Odborná anglič…","""'doc. Ing. Jan…","""1064""","""Paříková Anna,…"
"""KCH""","""BK214""",4226,"""Se""",2023,"""10.5.2024""","""10.5.2024""","""09:00""","""12:50""","""KCH""","""Odborný seminá…","""'doc. Ing. Jan…","""1064""","""Kadlečková Iva…"


In [ ]:
predmety_by_kat.filter(pl.col("zkratka") == "KTZI2").head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""KTZI2""",2023,"""Teoretické zák…","""Teoretické zák…","""N""","""A""",3,"""NE""","""""","""'RNDr. Martin …","""'RNDr. Martin …",[264],"""'RNDr. Martin …","""'RNDr. Martin …","[264, 267]","""'RNDr. Martin …","""'RNDr. Martin …","[264, 267]","""""","""""",null,"""""","""""","""""","""""","""KI/KTZI1""","""""","""""","""'Demel, J. Gra…","""""","""""","""""","""N""",6,"""HOD/SEM""",4,"""HOD/SEM""",2,"""HOD/SEM""","""Kurz je koncip…","""Zápočet""","""NE""","""Kombinovaná""","""Požadavky k zá…","""1. Rekurence a…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""


### warning, když je mezi vyučujícími někdo bez odpovídajícího typu rozvrhové akce

Pro každého učitele vypíšu předměty, u kterých má mít nějakou akci.

Takže budu mít dataframe, kde bude učitel, zkratka, a jeden sloupeček za každý typ akce, kde učitel bude muset mít aspoň 1 narozvrhovaný typ akce s daným předmětem, kde je hodnota `True`


In [ ]:
# vypíšu typy akcí
list(maly_rozvrh.select(pl.col("typAkceZkr").unique()))

[shape: (3,)
 Series: 'typAkceZkr' [str]
 [
 	"Se"
 	"Př"
 	"Cv"
 ]]

### garant předmětu by měl být někde mezi vyučujícími rozvrhových akcí a pokud má předmět přednášku, měl by být i mezi přednášejícími v rámci rozvrhových akcí

### Změny proti akreditacím.

### Porovnat revize předmětů

tj. parametry předmětů pro aktuální akademický rok a pro příští.

> jakou to má mít podobu?